# CIS CPS Homework 01 Power Grid State Estimation
## Introduction to Cyber-Physical Systems
## Leomar Duran
## 2022&ndash;02&ndash;23

First we import the state estimation module updated from Dr. Kant's original and the Pandapower Networks module.

In [ ]:
import state_est
import pandapower.networks as pn # for simple_four_bus_system
from scipy.stats import chi2
import matplotlib.pyplot as plt  # for plotting
import numpy as np               # for statistics
import math

Test running using the state estimate with all defaults.

In [ ]:
state_est.main()

Bad data found
Voltages: 
0    1.000000
1    1.000000
2    1.000000
3    0.987007
4    0.975472
5    1.003375
6    0.985645
7    0.996185
8    0.957621
Name: vm_pu, dtype: float64

 Voltage angles:  0    0.000000
1    9.668741
2    4.771073
3   -2.406644
4   -4.017264
5    1.925602
6    0.621545
7    3.799120
8   -4.349934
Name: va_degree, dtype: float64

########################################################################################################
>>> est_res_bus:
  vm_pu p_mw q_mvar
0     +    +      +
1     +    +      +
2     +    +      +
3     +    +      +
4     +    +      +
5     +    +      +
6     +    +      +
7     +    +      +
8     +    +      +
>>> est_res_trafo:
Empty DataFrame
Columns: [p_hv_mw, q_hv_mvar, p_lv_mw, q_lv_mvar, i_hv_ka, i_lv_ka]
Index: []
>>> est_res_trafo3w:
Empty DataFrame
Columns: [p_hv_mw, q_hv_mvar, p_mv_mw, q_mv_mvar, p_lv_mw, q_lv_mvar, i_hv_ka, i_mv_ka, i_lv_ka]
Index: []
>>> est_res_line:
  p_from_mw q_from_mvar p_to_mw q_to_mvar i_f

Run using `simple_four_bus_system()` and no perturbation.

In [ ]:
def no_perturbation(k):
    return 0
# def no_perturbation(k)

state_est.main(get_net=pn.simple_four_bus_system, bus_dvs_from_index=no_perturbation)

No bad data found
Done.


Inject bad data into simple four bus system. In order to do this, we write a function that returns a difference of $1\,\mathrm{V}$ if pass an index $1$ `lambda k: 1 if k == 1 else 0` and use it to increase the voltage of bus index $V_1$.

### Part C.)

In [ ]:
state_est.main(get_net=pn.simple_four_bus_system,
    bus_dvs_from_index=lambda k: 1 if k == 1 else 0, disp_chi2=True)

Bad data found
chi2_test on perturbed net2: True
Voltages: 
0    1.000000
1    0.996608
2    0.937760
3    0.902000
Name: vm_pu, dtype: float64

 Voltage angles:  0      0.000000
1   -150.208127
2   -149.007459
3   -148.184087
Name: va_degree, dtype: float64

########################################################################################################
>>> est_res_bus:
  vm_pu p_mw q_mvar
0     +    +      +
1     +    +      +
2     +    +      +
3     +    +      +
>>> est_res_trafo:
  p_hv_mw q_hv_mvar p_lv_mw q_lv_mvar i_hv_ka i_lv_ka
0       +         +       +         +       +       +
>>> est_res_trafo3w:
Empty DataFrame
Columns: [p_hv_mw, q_hv_mvar, p_mv_mw, q_mv_mvar, p_lv_mw, q_lv_mvar, i_hv_ka, i_mv_ka, i_lv_ka]
Index: []
>>> est_res_line:
  p_from_mw q_from_mvar p_to_mw q_to_mvar i_from_ka i_to_ka
0         +           +       +         +         +       +
1         +           +       +         +         +       +
Done.


As we can see from the returned voltages on bus $V_1$, the extra $1\,\mathrm{V}$ we added to make it about $1.96\,\mathrm{V}$ was removed so that it matches the other $3$ busses.

For the $\chi^2$ test, we have $4$ busses each with $2$ variables (magnitude and phase of power or voltage). However, the last phase is redundant as it is just $\frac{360}4^\circ$ more than the previous and $\frac{360}4^\circ$ less than the initial. Thus we have $k = (4\times2) - 1=7$ degrees of freedom. Additionally, we use the default probability of false positive for [`#chi2_analysis`](https://pandapower.readthedocs.io/en/v2.4.0/estimation.html#pandapower.estimation.chi2_analysis), `chi2_prob_false` $q = 0.05$. So $p = 1 - q = 0.95$, and the maximum allowed variance $F(0.95; 7) =$

In [ ]:
chi2.cdf(0.95, df=7)

0.004404710360513543

We were expecting to find data matching $\vec{V} := \begin{bmatrix}1 & 2 & 1 & 1\end{bmatrix}^\intercal$ with an average of $E[V] = 5/4$ and a variance $\operatorname{Var}(V) = 7/4 - (5/4)^2 = 3/16 = 0.1875$. However, variance $0.1875 > 0.0044 = F(0.95; 7)$. Thus the expected values failed the $\chi^2$ test and were removed.

Now, since the extra $1\,\mathrm{V}$ was removed from $V_1$, the resulting data was $\hat{V} := \begin{bmatrix}1.000000 & 0.996608 & 0.937760 & 0.937760\end{bmatrix}^\intercal$ with average $E[\hat{V}] = 0.968032$ and variance $\operatorname{Var}(\hat{V}) = 0.93800378525 - (0.968032)^2 = 0.000917832(226)$, and $0.000917832 \leq 0.00440471 = F(0.95; 7)$. Thus the resulting values do pass the $\chi^2$ test.

Moreover, for comparison to the expected values, the rates of error are $-0.225574$ for the average and $-0.930034$ for the standard deviation.

In [ ]:
expected = (1, 2, 1, 1)
result = (1.000000, 0.996608, 0.937760, 0.937760)

E_exp = np.mean(expected)
E_res = np.mean(result)
Var_exp = np.var(expected)
Var_res = np.var(result)
sigma_exp = math.sqrt(Var_exp)
sigma_res = math.sqrt(Var_res)

print(r'===expected===')
print('E\t', E_exp)
print('Var\t', Var_exp)
print(r'===result===')
print('E\t', E_res)
print('Var\t', Var_res)
print(r'===rate of error===')
print('E\t', (E_res - E_exp)/E_exp)
print('sigma\t', (sigma_res - sigma_exp)/sigma_exp)

===expected===
E	 1.25
Var	 0.1875
===result===
E	 0.968032
Var	 0.0009178321919999996
===rate of error===
E	 -0.2255744
sigma	 -0.9300349728507163
